In [223]:
import pandas as pd
import numpy as np

In [224]:
# 데이터 불러오기
trade_df = pd.read_csv('../trades.csv')
complex_df = pd.read_csv('../complexes.csv')
pyeong_df = pd.read_csv('../pyeongs.csv')

In [225]:
# pyeong 데이터에서 complexNo와  exclusiveArea가 동일한 행들을 모두 제거
pyeong_df.drop_duplicates(subset=['complexNo', 'exclusiveArea'], inplace=True)

pyeong_df[['complexNo', 'exclusiveArea']].value_counts().sort_values(ascending=False)

complexNo  exclusiveArea
1          59.13            1
           69.21            1
           84.73            1
3          84.92            1
           119.67           1
                           ..
134766     44.35            1
           46.38            1
134768     27.36            1
134728     40.47            1
134768     29.98            1
Length: 62975, dtype: int64

In [229]:
# pyeong과 complex 조인
pyeong_complex_df = pd.merge(
                        pyeong_df, 
                        complex_df, 
                        left_on='complexNo',
                        right_on='complexNo',
                        how='inner'
                    )

In [232]:
# completionYear 컬럼이 NaN인 행을 모두 제거
pyeong_complex_df.dropna(subset = ['completionYear'], inplace=True)
print(pyeong_complex_df)

       Unnamed: 0_x     id  complexNo  pyeongNo  supplyArea pyeongName  \
0                 0      1          1         1       82.29         82   
1             12174  12175          1         2       89.83         89   
2             23435  23436          1         4      103.32        103   
3             32898  32899          1         3      104.86        104   
4                 1      2         10         1       55.51         55   
...             ...    ...        ...       ...         ...        ...   
44406         12173  12174       9999         1       44.91         44   
44407         23434  23435       9999         2       48.38         48   
44408         32899  32900       9999         4       68.92        68A   
44409         40280  40281       9999         5       70.40        70B   
44410         45896  45897       9999         3       72.20         72   

       supplyPyeong pyeongName2  exclusiveArea  exclusivePyeong  ...  \
0             24.89          24        

In [233]:
pyeong_complex_df[['detailAddress', 'exclusiveArea', 'completionYear']].value_counts().sort_values(ascending=False)

detailAddress   exclusiveArea  completionYear
105             46.75          1993.0            4
                118.25         1993.0            3
                59.89          1993.0            3
                84.66          1993.0            2
                108.08         1993.0            2
                                                ..
항동지구5블록         76.95          2019.0            1
                84.87          2019.0            1
세곡2보금자리주택지구4블럭  101.40         2014.0            1
                101.18         2014.0            1
39-649          12.16          2013.0            1
Length: 44373, dtype: int64

In [236]:
pyeong_complex_df.drop_duplicates(subset=['detailAddress', 'exclusiveArea', 'completionYear'], inplace=True)

pyeong_complex_df[['detailAddress', 'exclusiveArea', 'completionYear']].value_counts().sort_values(ascending=False)

detailAddress  exclusiveArea  completionYear
1              41.58          1991.0            1
               84.92          2000.0            1
               68.76          1991.0            1
               71.78          1990.0            1
               176.40         1988.0            1
                                               ..
항동지구5블록        76.88          2019.0            1
               76.95          2019.0            1
               84.87          2019.0            1
항동공공주택지구6블록    116.99         2020.0            1
항동지구5블록        106.87         2019.0            1
Length: 44373, dtype: int64

In [240]:
# '실거래가 데이터'와 '단지데이터+평데이터' 조인
pyeong_complex_df['fk'] = pyeong_complex_df['detailAddress'] + ' ' \
                        + pyeong_complex_df['exclusiveArea'].astype('str') + ' ' \
                        + pyeong_complex_df['completionYear'].astype('int64').astype('str')

trade_df['fk'] = trade_df['detailAddress'] + ' ' \
                + trade_df['exclusiveArea'].astype('str') + ' ' \
                + trade_df['constructYear'].astype('str')

raw_train_df = pd.merge(trade_df, 
                        pyeong_complex_df, 
                        left_on='fk', 
                        right_on='fk',
                        how='left'
                       )

print(raw_train_df)

        Unnamed: 0     id_x      address_x detailAddress_x  detailAddressMain  \
0                0   999571  서울특별시 도봉구 방학동             713                713   
1                1   999572  서울특별시 강남구 개포동           655-2                655   
2                2   999573  서울특별시 강남구 개포동           655-2                655   
3                3   999574  서울특별시 강남구 개포동           658-1                658   
4                4   999575  서울특별시 강남구 개포동           658-1                658   
...            ...      ...            ...             ...                ...   
999565      999565  1999136  서울특별시 중랑구 중화동            11-7                 11   
999566      999566  1999137  서울특별시 중랑구 중화동            11-7                 11   
999567      999567  1999138  서울특별시 중랑구 중화동            11-7                 11   
999568      999568  1999139  서울특별시 중랑구 중화동            11-7                 11   
999569      999569  1999140  서울특별시 중랑구 중화동            11-7                 11   

        detailAddressSub   

In [183]:
# 불필요한 컬럼 제거
raw_train_df = raw_train_df[['id_x', 'detailAddress_x', 'complexName_x', 'exclusiveArea_x', 'contactDate', 'price', 'floor', 'constructYear',
       'roadAddress_x', 'complexNo', 'pyeongNo', 'supplyArea', 'supplyPyeong', 'exclusiveArea_y', 'exclusivePyeong',
       'householdCountByPyeong', 'realEstateTypeCode_x', 'exclusiveRate', 'entranceType', 'roomCnt', 'bathroomCnt',
       'complexName_y', 'cortarNo',  'detailAddress_y', 'roadAddress_y', 'latitude', 'longitude', 'totalHouseholdCount', 
       'totalLeaseHouseholdCount', 'highFloor', 'lowFloor', 'completionYear', 'completionMonth', 'totalDongCount',
       'maxSupplyArea', 'minSupplyArea', 'dealCount', 'rentCount', 'batlRatio', 'btlRatio', 'parkingCountByHousehold', 
       'constructionCompanyName', 'heatMethodTypeCode', 'heatFuelTypeCode', 'address_y']]

raw_train_df.columns = ['id', 'detailAddress', 'complexName', 'exclusiveArea', 'contactDate', 'price', 'floor', 'constructYear',
       'roadAddress', 'complexNo', 'pyeongNo', 'supplyArea', 'supplyPyeong', 'exclusiveArea', 'exclusivePyeong',
       'householdCountByPyeong', 'realEstateTypeCode', 'exclusiveRate', 'entranceType', 'roomCnt', 'bathroomCnt',
       'complexName', 'cortarNo',  'detailAddress', 'roadAddress', 'latitude', 'longitude', 'totalHouseholdCount', 
       'totalLeaseHouseholdCount', 'highFloor', 'lowFloor', 'completionYear', 'completionMonth', 'totalDongCount',
       'maxSupplyArea', 'minSupplyArea', 'dealCount', 'rentCount', 'batlRatio', 'btlRatio', 'parkingCountByHousehold', 
       'constructionCompanyName', 'heatMethodTypeCode', 'heatFuelTypeCode', 'address']

In [242]:
raw_train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 999570 entries, 0 to 999569
Data columns (total 83 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   Unnamed: 0                    999570 non-null  int64  
 1   id_x                          999570 non-null  int64  
 2   address_x                     999570 non-null  object 
 3   detailAddress_x               999418 non-null  object 
 4   detailAddressMain             999570 non-null  int64  
 5   detailAddressSub              999570 non-null  int64  
 6   complexName_x                 999570 non-null  object 
 7   exclusiveArea_x               999570 non-null  float64
 8   contactDate                   999570 non-null  object 
 9   price                         999570 non-null  int64  
 10  floor                         999570 non-null  int64  
 11  constructYear                 999570 non-null  int64  
 12  roadAddress_x                 999570 non-nul